Following https://realpython.com/beautiful-soup-web-scraper-python/
and https://www.scrapingbee.com/blog/selenium-python/


install webdriver for Chrome from https://chromedriver.chromium.org/downloads

in your python environment install selenium https://selenium-python.readthedocs.io/installation.html

to identify xpath location of relevant content can use https://selectorshub.com/selectorshub/

In [24]:
import json
import pandas as pd

import os

from selenium import webdriver
from selenium.webdriver.common.by import By

import pprint

In [25]:
config_json = "scraping_config.json"
institution_json = "institution.json"

config_file = open(config_json)

config = json.load(config_file)
config_file.close()

institutions = config['institutions']
driver_path = config['driver_path']

print(institutions)

pp = pprint.PrettyPrinter(indent=4)

['Durham_UK', 'Glasgow_UK']


In [6]:
# See https://stackoverflow.com/questions/43085744/parsing-robots-txt-in-python
import urllib.robotparser as urobot
import urllib.request
from urllib.parse import urlparse

rp = urobot.RobotFileParser()
# TODO check the robot file

In [52]:
driver = webdriver.Chrome(executable_path=driver_path)

try:
    for institution_name in institutions:
        path = institution_name

        institution_file = open(os.path.join(path, institution_json))

        institution_config = json.load(institution_file)
        institution_file.close()
        pp.pprint(institution_config)
        modules_df = pd.DataFrame(columns=['institution', 'cohort', 'module', 'title', 'content'], dtype="string")
    #modules_df = pd.DataFrame(dtype="string")


        url = institution_config['scrapeURL']
        
        print ("url ", url)
        
        urlp = urlparse(url)
    # robots.txt doesn't seem to work
        """
        robots_url = urlp._replace(path="/robots.txt", fragment="", query="").geturl()
        driver.get(robots_url)
        rp.read()
        """

        xpaths = institution_config['XPath']
        cohorts = institution_config['cohorts']
        for cohort in cohorts:
            modules = cohort['modules']
            for module in modules:
                module_url = url.replace("%MODULE%", module)
    #            if not rp.can_fetch("*", module_url):
    #                print("robots.txt denies access to " + module_url)
    #                continue
                driver.get(module_url)
                content = ""
                content_paths = ['title', 'summary', 'content', 'ilo']
                for content_path in content_paths:
                    try:
                        content_elts = driver.find_elements(By.XPATH, xpaths[content_path])
                    except Exception:
                        continue
                    content += "<h2>" + content_path + "</h2>"
                    for elt in content_elts:
                        content += elt.get_attribute('innerHTML')
                title = ""
                title_elts = driver.find_elements(By.XPATH, xpaths['title'])
                for elt in title_elts:
                    title += elt.get_attribute('innerHTML')

                cohort_descriptor = cohort['programme'] + " " + cohort['level'] + " " + cohort['academic_year']
                new_row = {"institution": institution_config['institution'],
                           "cohort":  cohort_descriptor,
                          "module": module,
                          "title": title,
                          "content": content}
                modules_df = modules_df.append(new_row, ignore_index=True)
                modules_df.to_csv(os.path.join(path,'modules.csv'), index= False)

    driver.quit()
except OSError as err:
    print("OS error:", err)
    
# except Exception as err:
#     print(f"Unexpected {err=}, {type(err)=}")
#     driver.quit()
 


/var/folders/my/th4q3hxs0b7c1t1qlv0j2dtm0000gp/T/ipykernel_85089/3803288518.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


{   'XPath': {   'content': "//*[@id='fhcontent']/following-sibling::ul[1]",
                 'ilo': "//div[normalize-space()='Subject-specific "
                        "Skills:']/following-sibling::ul[1]",
                 'title': '//h3'},
    'cohorts': [   {   'academic_year': '2022',
                       'level': '3',
                       'modules': [   'COMP3477',
                                      'COMP3421',
                                      'COMP3487',
                                      'COMP3491',
                                      'COMP3517',
                                      'COMP3527'],
                       'programme': 'CS'}],
    'institution': 'Durham',
    'scrapeURL': 'https://apps.dur.ac.uk/faculty.handbook/2022/UG/module/%MODULE%'}
url  https://apps.dur.ac.uk/faculty.handbook/2022/UG/module/%MODULE%


/var/folders/my/th4q3hxs0b7c1t1qlv0j2dtm0000gp/T/ipykernel_85089/3803288518.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modules_df = modules_df.append(new_row, ignore_index=True)
/var/folders/my/th4q3hxs0b7c1t1qlv0j2dtm0000gp/T/ipykernel_85089/3803288518.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modules_df = modules_df.append(new_row, ignore_index=True)
/var/folders/my/th4q3hxs0b7c1t1qlv0j2dtm0000gp/T/ipykernel_85089/3803288518.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modules_df = modules_df.append(new_row, ignore_index=True)
/var/folders/my/th4q3hxs0b7c1t1qlv0j2dtm0000gp/T/ipykernel_85089/3803288518.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

{   'XPath': {   'content': "//*[@id='fhcontent']/following-sibling::ul[1]",
                 'ilo': '',
                 'summary': "//span[@id='_%MODULE%']",
                 'title': '/html[1]/body[1]/div[4]/div[2]/div[1]/div[1]/div[1]/main[1]/div[1]/h2[1]'},
    'cohorts': [   {   'academic_year': '2022',
                       'level': '3',
                       'modules': ['COMPSCI4021'],
                       'programme': 'CS'}],
    'institution': 'Glasgow_UK',
    'scrapeURL': 'https://www.gla.ac.uk/coursecatalogue/course/?code=%MODULE%'}
url  https://www.gla.ac.uk/coursecatalogue/course/?code=%MODULE%


/var/folders/my/th4q3hxs0b7c1t1qlv0j2dtm0000gp/T/ipykernel_85089/3803288518.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modules_df = modules_df.append(new_row, ignore_index=True)


In [50]:
driver.quit()